In [50]:
import keras
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input, decode_predictions

from keras.applications.resnet import ResNet101
from keras.preprocessing import image

import numpy as np

from collections import defaultdict

In [35]:
# instantiate ResNet50 model
model50 = ResNet50(weights='imagenet')

# instantiate ResNet101 model
model101 = ResNet101(weights='imagenet')

In [36]:
IMG_PATH = 'bernie.png'
LABELS_PATH = 'labels.txt'
SIDE = 224
ALPHA = 0.7

with open(LABELS_PATH) as f:
    labels = f.read()
    labels = eval(labels)

In [37]:
resnet101_target = 'printer'
resnet50_target = 'gazzelle'

In [38]:
base_img = image.load_img(IMG_PATH)
base_img = image.img_to_array(base_img)
assert base_img.shape == (SIDE, SIDE, 3)

In [39]:
def get_rnd_noise_img():
    return np.random.uniform(0, 255, (SIDE, SIDE, 3))

def mix_images(img1, img2, alpha):
    return alpha * img1 + (1 - alpha) * img2

In [52]:
from re import T

count = defaultdict(int)

for i in range(1000):
    noise = get_rnd_noise_img()
    img = mix_images(base_img, noise, ALPHA)
    pred50 = model50.predict(preprocess_input(img[None]))
    pred101 = model101.predict(preprocess_input(img[None]))
    pred50 = decode_predictions(pred50, top=1)[0][0][1].replace('_', ' ')
    pred101 = decode_predictions(pred101, top=1)[0][0][1].replace('_', ' ')
    count[pred50] += 1
    count[pred101] += 1
    if pred101 == resnet101_target:
        print(pred101)
        image.save_img('found101.png', img)
    if pred50 == resnet50_target:
        print(pred50)
        image.save_img('found50.png', img)
    if pred50 == resnet50_target and pred101 == resnet101_target:
        # save to file
        print('Found image')
        image.save_img('found.png', img)
        break

print(count)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━